In [2]:
#!pip install pandas_profiling

In [3]:
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import  ElasticNet, Lasso, LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

import joblib

import warnings
warnings.filterwarnings('ignore')

c:\Users\Kankash Rayaneh\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
c:\Users\Kankash Rayaneh\AppData\Local\Programs\Python\Python310\lib\site-packages\visions\backends\shared\nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:
C:\Users\Kankash Rayaneh\AppData\Local\Temp\ipykernel_18860\106047948

In [4]:
df = pd.read_csv('tehranhouses.csv')
df.head()

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,True,True,True,Shahran,1850000000,61666.67
1,60,1,True,True,True,Shahran,1850000000,61666.67
2,79,2,True,True,True,Pardis,550000000,18333.33
3,95,2,True,True,True,Shahrake Qods,902500000,30083.33
4,123,2,True,True,True,Shahrake Gharb,7000000000,233333.33


In [5]:
df.drop('Price(USD)', axis=1, inplace=True)

In [6]:
#pandas_profiling.ProfileReport(df)

In [7]:
df['Area'] = df['Area'].str.replace(',', '').astype(float)

In [8]:
df = df.drop_duplicates()

In [9]:
df.dropna(inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3248 entries, 0 to 3478
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Area       3248 non-null   float64
 1   Room       3248 non-null   int64  
 2   Parking    3248 non-null   bool   
 3   Warehouse  3248 non-null   bool   
 4   Elevator   3248 non-null   bool   
 5   Address    3248 non-null   object 
 6   Price      3248 non-null   int64  
dtypes: bool(3), float64(1), int64(2), object(1)
memory usage: 136.4+ KB


In [11]:
def upper(x):
    Q1 = np.percentile(x, 25)
    Q3 = np.percentile(x, 75)
    IQR = Q3 - Q1
    upper = Q3 + 1.5 * IQR
    
    return upper

upper_area = upper(df['Area'])
upper_price = upper(df['Price'])

df= df[df['Area'] < upper_area]
df= df[df['Price'] < upper_price]

In [12]:
df.describe()

,Area,Room,Price
count,2914.000000,2914.000000,2.914000e+03
mean,91.064859,1.958476,3.502930e+09
std,31.484197,0.644493,2.960032e+09
min,30.000000,0.000000,3.600000e+06
25%,67.000000,2.000000,1.260000e+09
50%,87.000000,2.000000,2.565000e+09
75%,110.000000,2.000000,4.700000e+09
max,198.000000,5.000000,1.330000e+10


In [13]:
import plotly.express as px

fig = px.histogram(df, x='Price', histfunc='count')
fig.show()

In [14]:
import plotly.express as px

fig = px.histogram(df, x='Area', histfunc='count')
fig.show()

In [15]:
fig = px.histogram(df, x='Area', y= 'Price', histfunc='count')
fig.show()

In [16]:
fig = px.bar(df, x='Area', y='Price')
fig.show(renderer="iframe")

In [17]:
for i in ['Room', 'Parking', 'Warehouse', 'Elevator']:
  fig= px.pie(df, names= i, title= f'distribution of {i}')
  fig.show()

In [18]:
import plotly.graph_objects as go

frequencies = df['Address'].value_counts().head(8)
fig = go.Figure([go.Bar(x=frequencies.index, y=frequencies)])
fig.show()

In [19]:
#pip install pycaret

In [20]:
#from pycaret.regression import *

In [21]:
#setup(df, target= 'Price', session_id=18)

In [22]:
#models()

In [23]:
#compare_models()

In [24]:
df = pd.get_dummies(df, columns= ['Address'])

df.replace(True, 1, inplace=True)
df.replace(False, 0, inplace= True)

df

,Area,Room,Parking,Warehouse,Elevator,Price,Address_Abazar,Address_Abbasabad,Address_Abuzar,Address_Afsarieh,...,Address_Waterfall,Address_West Ferdows Boulevard,Address_West Pars,Address_Yaftabad,Address_Yakhchiabad,Address_Yousef Abad,Address_Zafar,Address_Zaferanieh,Address_Zargandeh,Address_Zibadasht
0,63.0,1,1,1,1,1850000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,60.0,1,1,1,1,1850000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,79.0,2,1,1,1,550000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,95.0,2,1,1,1,902500000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,123.0,2,1,1,1,7000000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3473,63.0,1,1,1,0,1890000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3474,86.0,2,1,1,1,3500000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3475,83.0,2,1,1,1,6800000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3477,105.0,2,1,1,1,5600000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
X = df.drop(columns = 'Price')
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [26]:
from catboost import CatBoostRegressor

cb= CatBoostRegressor(random_state= 1)
param_cb= {'iterations':[100,1000], 'learning_rate':[0.001, 0.01, 0.1]}

grid = GridSearchCV(cb, 
                        param_grid = param_cb, 
                        refit = True, 
                        cv = KFold(shuffle = True, random_state = 1), 
                        n_jobs = -1)
grid.fit(X_train, y_train)
y_train_pred = grid.predict(X_train)
y_test_pred = grid.predict(X_test)
    
train_score= r2_score(y_train, y_train_pred)
test_score= r2_score(y_test, y_test_pred)

print(train_score)
print(test_score)

0:	learn: 2853338689.3902597	total: 135ms	remaining: 2m 14s
1:	learn: 2720631311.6644788	total: 140ms	remaining: 1m 9s
2:	learn: 2614021046.5076675	total: 143ms	remaining: 47.4s
3:	learn: 2498439208.3461719	total: 145ms	remaining: 36.1s
4:	learn: 2393039205.0989132	total: 147ms	remaining: 29.2s
5:	learn: 2323055240.8742514	total: 149ms	remaining: 24.6s
6:	learn: 2243691353.4984508	total: 150ms	remaining: 21.3s
7:	learn: 2181380838.5189857	total: 152ms	remaining: 18.8s
8:	learn: 2129406396.4381640	total: 153ms	remaining: 16.9s
9:	learn: 2081124901.1695001	total: 155ms	remaining: 15.3s
10:	learn: 2031677786.9267986	total: 156ms	remaining: 14.1s
11:	learn: 1981627122.2776575	total: 159ms	remaining: 13.1s
12:	learn: 1943896959.7517476	total: 160ms	remaining: 12.2s
13:	learn: 1907036766.2424533	total: 162ms	remaining: 11.4s
14:	learn: 1882955087.0121913	total: 163ms	remaining: 10.7s
15:	learn: 1848746421.0569890	total: 165ms	remaining: 10.1s
16:	learn: 1824466641.4744956	total: 166ms	remain

In [27]:
def preprocessor(df2):
    df2 = pd.get_dummies(df2, columns=['Address'])
    df2 = df2.replace({'True': 1, 'False': 0})
    df2 = df2.reindex(columns=X_train.columns, fill_value=0)
    return df2

In [28]:
def predictprice(Area,Room,Parking,Warehouse,Elevator,Address): 
    row = np.array([Area,Room,Parking,Warehouse,Elevator,Address]) 
    X = pd.DataFrame([row], columns = ['Area','Room','Parking','Warehouse','Elevator','Address'])
    print(X)
    p_x= preprocessor(X)
    prediction = grid.predict(p_x)
    print(prediction)

In [29]:
joblib.dump(grid, 'catboost.joblib')

['catboost.joblib']